In [1]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext Cython

In [9]:
%%cython
cimport cython
from libc.stdlib cimport malloc, free
from cython cimport sizeof
#import numpy as np
#cimport numpy as np



ctypedef struct f_state:
    int* state
    int size

cdef void free_fermions(f_state fstate) nogil:
    free(fstate.state)


cdef f_state create_fermions(str state, int space_size):
    cdef int* state_array = <int*>malloc(space_size*sizeof(int))
    if state_array is NULL:
        raise MemoryError("Failed to allocate memory for state_array")
    
    for i in range(space_size):
        if state[i] not in ('0', '1'):
            raise ValueError("spinless fermion states can only have 1 occupation or none 0")
        state_array[i] = <int>state[i]-48 #minus 48 because of ascii encoding from str to int. 0 is 48 in ascii. 1 is 49
    cdef f_state fstate
    fstate.state = state_array
    fstate.size = space_size
    return fstate

cdef int* create_explicit_fermions(f_state in_state):
    #start by allocating the vector
    cdef int total_size = 2**in_state.size
    #malloc this to mem
    #....to be completed
    cdef int tmp_dimension
    cdef int index
    if in_state.state[0] == 0:
        index = 0
    elif in_state.state[0] == 1:
        index = 1
    if (in_state.size > 1):
        for i in range(in_state.size):
            tmp_dimension = 2**(i+1)
            if (in_state.state[i+1] == 0):
                index = index
            elif (in_state.state[i+1] == 1):
                index = index + tmp_dimension
    cdef int* result = <int*>malloc(total_size*sizeof(int))
    for i in range(total_size):
        result[i] = 0
    result[index] = 1
    return result

    








#test unit 2

cdef str fermions = "010010"
cdef int fsize = 6

print("\nthe state in diract notation")
cdef f_state fermion_state = create_fermions(fermions, fsize)
for i in range(fsize):
    print(fermion_state.state[i])

print("\nthe state in an explicit Fock space vector\n")
cdef int* fermion_vector = create_explicit_fermions(fermion_state)
for i in range(2**fsize):
    print(fermion_vector[i])
    
free_fermions(fermion_state)
free(fermion_vector)





the state in diract notation
0
1
0
0
1
0

the state in an explicit Fock space vector

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
